In [1]:
import pandas as pd
import numpy as np
import surprise
import apyori

In [6]:
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/apple/.surprise_data/ml-100k


In [7]:
testSet = trainingSet.build_testset()

In [17]:
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()
num_users = trainingSet.n_users
print(f"Количество пользователей в trainingSet: {num_users}")

Количество пользователей в trainingSet: 943


In [18]:
num_items = trainingSet.n_items
num_items

1682

In [27]:
import surprise
from surprise import KNNBasic, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy




trainset, testset = train_test_split(data, test_size=0.25)

# Тренировка модели KNNBasic
algo_knn = KNNBasic()
algo_knn.fit(trainset)

# Тренировка модели SVD
algo_svd = SVD()
algo_svd.fit(trainset)

# Предсказания на тестовом наборе для обеих моделей
predictions_knn = algo_knn.test(testset)
predictions_svd = algo_svd.test(testset)

# Индекс, для которого мы хотим получить предсказание
index = 8  # Замените на нужный индекс

# Получение предсказания под -ым индексом
pred_knn = predictions_knn[index]
pred_svd = predictions_svd[index]

# Вывод информации о предсказаниях
print(f"KNNBasic: Пользователь {pred_knn.uid} -> Рекомендованный фильм {pred_knn.iid}")
print(f"SVD: Пользователь {pred_svd.uid} -> Рекомендованный фильм {pred_svd.iid}")

# Вывод id рекомендованного фильма для KNNBasic
print(f"ID рекомендованного фильма (KNNBasic): {pred_knn.iid}")

# Вывод id рекомендованного фильма для SVD
print(f"ID рекомендованного фильма (SVD): {pred_svd.iid}")
# Предсказания на тестовом наборе для обеих моделей
predictions_knn = algo_knn.test(testset)
predictions_svd = algo_svd.test(testset)

# Расчет RMSE для обеих моделей
rmse_knn = accuracy.rmse(predictions_knn, verbose=False)
rmse_svd = accuracy.rmse(predictions_svd, verbose=False)

# Сравнение и выбор лучшего результата
best_rmse = min(rmse_knn, rmse_svd)

# Округление до двух знаков после точки
best_rmse_rounded = round(best_rmse, 2)

# Вывод RMSE для лучшего алгоритма
print(f"Лучшее значение RMSE: {best_rmse_rounded}")

Computing the msd similarity matrix...
Done computing similarity matrix.
KNNBasic: Пользователь 521 -> Рекомендованный фильм 246
SVD: Пользователь 521 -> Рекомендованный фильм 246
ID рекомендованного фильма (KNNBasic): 246
ID рекомендованного фильма (SVD): 246
Лучшее значение RMSE: 0.94


In [28]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df.head()

,user_id,movie_id,rating,3
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [29]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

# Отфильтруем строки, где рейтинг >= 4
df_filtered = df[df['rating'] >= 4]

# Группируем по user_id и объединяем movie_id в строку с пробелом в качестве разделителя
user_movie_strings = df_filtered.groupby('user_id')['movie_id'].apply(lambda x: ' '.join(map(str, x)))

# Получаем строку для user_id 100
user_100_movies = user_movie_strings.get(100, "")


# Вывод результата
print(f"Строка для user_id 100: {user_100_movies}")

Строка для user_id 100: 4 435 153 310 258 319 887 211 325 202 8


/var/folders/07/56f70q6n70d3v657jsy7fjgr0000gn/T/ipykernel_61018/1049805414.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  user_100_movies = user_movie_strings.get(100, "")


In [30]:
import surprise
import pandas as pd
from apyori import apriori

# Загрузка встроенного набора данных
data = surprise.Dataset.load_builtin("ml-100k")

# Создаем DataFrame из raw_ratings
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

# Отфильтруем строки, где рейтинг >= 4
df_filtered = df[df['rating'] >= 4]

# Группируем фильмы по пользователям, формируя транзакции
transactions = df_filtered.groupby('user_id')['movie_id'].apply(list).tolist()

# Применение алгоритма Apriori с заданными параметрами
rules = apriori(transactions, min_support=0.2, min_confidence=0.3, min_lift=2, min_length=2)

# Преобразование результата в список
rules_list = list(rules)

# Подсчет количества правил
num_rules = len(rules_list)

# Вывод результата
print(f"Количество ассоциативных правил: {num_rules}")


Количество ассоциативных правил: 10


In [32]:
import surprise
import pandas as pd
from apyori import apriori
from collections import Counter

# Загрузка встроенного набора данных
data = surprise.Dataset.load_builtin("ml-100k")

# Создаем DataFrame из raw_ratings
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

# Отфильтруем строки, где рейтинг >= 4
df_filtered = df[df['rating'] >= 4]

# Группируем фильмы по пользователям, формируя транзакции
transactions = df_filtered.groupby('user_id')['movie_id'].apply(list).tolist()

# Применение алгоритма Apriori с заданными параметрами
rules = apriori(transactions, min_support=0.2, min_confidence=0.3, min_lift=2, min_length=2)

# Преобразование результата в список
rules_list = list(rules)

# Извлечение всех фильмов из правил
films = []
for rule in rules_list:
    for stat in rule.ordered_statistics:
        films.extend(list(stat.items_base))
        films.extend(list(stat.items_add))

# Подсчет частоты появления каждого фильма
film_counter = Counter(films)

# Поиск фильма, который встречается чаще всего
most_common_film = film_counter.most_common(1)[0]  # Получаем tuple (film_id, frequency)

# Вывод результата
print(f"ID фильма, который чаще всего встречается в правилах: {most_common_film[0]}")


ID фильма, который чаще всего встречается в правилах: 174
